In [ ]:
import pandas as pd
import numpy as np

import plotly
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import losses as losses

from torch import nn
import torch

In [ ]:
# SCALE_DATASET = None
SCALE_DATASET = "minmax"
# SCALE_DATASET = "standard"

In [ ]:
dataset = torch.tensor(np.array([[91.7600], [30.5500], [87.3800], [91.6200], [100.0000], [64.9000], [39.0900], [38.3100], [7.6700], [99.3700], [11.3200], [65.3300], [81.6600], [0.0000], [4.2700], [63.6500], [88.3800], [44.1100], [67.0900], [89.7600], [69.4400], [27.1100], [15.3300], [30.3800], [77.5100], [87.0500], [97.3200], [14.3900], [91.0400], [97.8900], [42.6400], [37.0100], [78.3100], [53.9000], [84.9300], [90.3000], [65.4800], [49.8300], [72.5100], [45.1100], [95.7400], [87.6700], [5.9400], [73.6000], [4.3900], [12.0700], [88.0900], [48.4700], [92.4700], [85.1400], [68.9600], [53.8300], [56.2300], [24.3400], [66.6600], [47.6100], [93.2500], [44.7700], [91.1700], [73.6200], [20.0600], [31.3500], [75.9000], [36.9300], [53.6900], [97.5100], [4.1400], [56.8300], [44.0700], [72.3200], [83.8400], [67.9500], [0.5200], [30.5400], [91.4000], [56.9700], [29.1800], [17.5300], [92.8300], [0.1300], [98.8000], [14.9800], [39.2800], [39.1900], [29.2300], [26.8700], [32.5200], [32.6200], [80.6300], [42.5400], [71.5200], [86.8000], [64.8900], [47.4900], [26.0700], [38.2700], [35.8200], [70.9800], [25.0400], [10.0000]]), dtype=torch.float32)
if SCALE_DATASET == "standard":
    dataset_mean = dataset.mean()
    dataset_std = dataset.std()
    dataset = (dataset - dataset_mean) / dataset_std
if SCALE_DATASET == "minmax":
    dataset_min = dataset.min()
    dataset_max = dataset.max()
    dataset = (dataset - dataset_min) / (dataset_max - dataset_min)
train_dataset = dataset[:80]
test_dataset = dataset[80:]
test_dataset

In [ ]:
awg_diff_train = torch.mean(torch.abs(train_dataset[1:] - train_dataset[:-1]))
print(awg_diff_train)
avg_sq_train_diff = torch.mean(torch.square(train_dataset[1:] - train_dataset[:-1]))
print(avg_sq_train_diff)
mean_train = torch.mean(train_dataset)
print(mean_train)



In [ ]:
predicted = torch.tensor(np.arange(-100, 121).reshape(-1,1), dtype=float)
print(predicted)
if SCALE_DATASET == "standard":
    predicted = (predicted - dataset_mean) / dataset_std
if SCALE_DATASET == "minmax":
    predicted = (predicted - dataset_min) / (dataset_max - dataset_min)
real = torch.tensor(np.full((len(predicted)),test_dataset[-1]).reshape(-1,1), dtype=float)
index = real - predicted


In [ ]:
def draw_loss(loss, func_name, predicted_example=130, **kwargs):
    loss_output = loss(predicted, real, **kwargs)

    loss_output_df = pd.DataFrame(
        data=np.concatenate(
            (
                index.reshape(-1).numpy().reshape(-1, 1),
                loss_output.reshape(-1).numpy().reshape(-1, 1),
            ),
            axis=1,
        ),
        columns=["error", func_name],
    )
    fig = px.line(loss_output_df, x=loss_output_df.columns[0], y=func_name, title=func_name)

    if "angle_loss" == func_name:
        data = {
            "x": [awg_diff_train, awg_diff_train, 0],
            "y": [
                test_dataset[-1][0],
                predicted[predicted_example][0],
                test_dataset[-2][0],
            ],
            "color": ["Target", "Predicted", "Previous"],  # Use descriptive names
            "info": ["", f"Predicted, error:{index[predicted_example][0]}", ""],
        }

        df = pd.DataFrame(data)

        fig_data = px.scatter(
            df, x="x", y="y", color="color", title="situation", hover_data=["info"]
        )
        ratio = float(kwargs["avg_diff"].numpy())
        fig_data.update_layout(
            xaxis=dict(scaleanchor="y", scaleratio=ratio),
            # xaxis=dict(scaleanchor="y", scaleratio=1),
            yaxis=dict(scaleanchor="x", scaleratio=1),
        )

        fig_data.add_scatter(
            x=[kwargs["avg_diff"], 0],  # x-coordinates of the line
            y=[predicted[predicted_example][0], test_dataset[-2][0]],  # y-coordinates of the line
            mode='lines',
            line=dict(color='black', width=1),  # Customize color and width
            name=""
        )
        fig_data.add_scatter(
            x=[kwargs["avg_diff"], 0],  # x-coordinates of the line
            y=[test_dataset[-1][0], test_dataset[-2][0]],  # y-coordinates of the line
            mode='lines',
            line=dict(color='black', width=1),  # Customize color and width
            name=""
        )

        fig.add_scatter(
            x=[loss_output_df.iloc[predicted_example]["error"]],  # Replace with your x coordinate
            y=[loss_output_df.iloc[predicted_example]["angle_loss"]],  # Replace with your y coordinate
            mode='markers',
            marker=dict(color='green', size=10),
            name='situation'
        )


        return fig, fig_data
    return fig

### Angle loss graph
There are 2 graphs below. First one shows graph of loss with one point. You can chane position of this point by changing value of predicted_example (from 0 to 220). This will change the secong graph, that describe real situation.

In [ ]:
loss_graph, situation_graph = draw_loss(losses.angle_loss, "angle_loss", predicted_example = 140, target_minus1 = test_dataset[-2], avg_diff = awg_diff_train, reduction = "none")
loss_graph.show()
situation_graph.show()

In [ ]:
draw_loss(losses.mae, "mae", reduction = "none")

In [ ]:
draw_loss(losses.mse, "mse", reduction = "none")

In [ ]:
draw_loss(losses.mase, "mase", train_avg_diff = awg_diff_train, reduction = "none")

In [ ]:
draw_loss(losses.rmsse, "rmsse", avg_sq_train_diff = avg_sq_train_diff, reduction = "none")

In [ ]:
draw_loss(losses.poisson, "poisson", reduction = "none")

In [ ]:
draw_loss(losses.mbe, "mbe", reduction = "none")

In [ ]:
draw_loss(losses.rae, "rea", dataset_mean = mean_train, reduction = "none")

In [ ]:
draw_loss(losses.rse, "rse", dataset_mean = mean_train, reduction = "none")

In [ ]:
draw_loss(losses.mape, "mape", reduction = "none")

In [ ]:
draw_loss(losses.rmse, "rmse", reduction = "none")

In [ ]:
draw_loss(losses.msle, "msle", reduction = "none")

In [ ]:
draw_loss(losses.rmsle, "rmsle", reduction = "none")

In [ ]:
draw_loss(losses.nrmse, "nrmse", norm_factor = mean_train , reduction = "none")

In [ ]:
draw_loss(losses.nrmse, "nrmse", norm_factor = mean_train , reduction = "none")

In [ ]:
draw_loss(losses.rrmse, "rrmse", reduction = "none")

In [ ]:
draw_loss(losses.huber, "huber_loss", reduction = "none", delta = mean_train/2)

In [ ]:
draw_loss(losses.logCosh, "logCosh", reduction = "none")

In [ ]:
draw_loss(losses.kernelMSE, "kernelMSE", reduction = "none")

In [ ]:
draw_loss(losses.quantile, "quantile", q = 0.75, reduction = "none")

In [ ]:
draw_loss(losses.quantile, "quantile", q = 0.25, reduction = "none")

In [ ]:
loss_values = [
    losses.huber(predicted, real, reduction = "none", delta = mean_train/2),
    losses.kernelMSE(predicted, real, reduction = "none"),
    losses.logCosh(predicted, real, reduction = "none"),
    losses.mae(predicted, real, reduction = "none"),
    losses.mape(predicted, real, reduction = "none"),
    losses.mase(predicted, real, train_avg_diff = awg_diff_train, reduction = "none"),
    losses.mbe(predicted, real, reduction = "none"),
    losses.mse(predicted, real, reduction = "none"),
    losses.msle(predicted, real, reduction = "none"),
    losses.nrmse(predicted, real, norm_factor = mean_train , reduction = "none"),
    losses.poisson(predicted, real, reduction = "none"),
    losses.quantile(predicted, real, q = 0.25, reduction = "none"),
    losses.quantile(predicted, real, q = 0.75, reduction = "none"),
    losses.rae(predicted, real, dataset_mean = mean_train, reduction = "none"),
    losses.rrmse(predicted, real, reduction = "none"),
    losses.rse(predicted, real, dataset_mean = mean_train, reduction = "none"),
    losses.rmse(predicted, real, reduction = "none"),
    losses.rmsle(predicted, real, reduction = "none"),
    losses.rmsse(predicted, real, avg_sq_train_diff = avg_sq_train_diff, reduction = "none"),
    losses.angle_loss(predicted, real, target_minus1 = test_dataset[-2], avg_diff = awg_diff_train, reduction = "none"),
]

In [ ]:
loss_names = [
    "Huber Loss", 
    "KernelMSE", 
    "Log-Cosh Loss", 
    "MAE", 
    "MAPE", 
    "MASE", 
    "MBE", 
    "MSE", 
    "MSLE", 
    "NRMSE", 
    "Poisson Loss", 
    "Quantile Loss 0.25", 
    "Quantile Loss 0.75", 
    "RAE", 
    "RRMSE", 
    "RSE", 
    "RMSE", 
    "RMSLE", 
    "RMSSE", 
    "Angle Loss"
]

In [ ]:
def draw_losses(loss_values, loss_names):
    
    # Create a 7x3 grid of subplots
    fig = make_subplots(
        rows=4, 
        cols=5, 
        subplot_titles=loss_names,  # Titles for each subplot
        vertical_spacing=0.05
    )

    # Add a trace to each subplot
    for i in range(1, 21):  # 21 subplots
        row = (i - 1) // 5 + 1  # Calculate the row number
        col = (i - 1) % 5 + 1   # Calculate the column number
        
        
        loss_output = loss_values[i-1]
        loss_output_df = pd.DataFrame(
            data=np.concatenate(
                (
                    index.reshape(-1).numpy().reshape(-1, 1),
                    loss_output.reshape(-1).numpy().reshape(-1, 1),
                ),
                axis=1,
            ),
            columns=["error", loss_names[i-1]],
        )
        # print(loss_output_df)
        # px.line(loss_output_df, x=loss_output_df.columns[0], y=loss_names[i-1], title=loss_names[i-1])
        # Add a simple scatter plot to each subplot

        fig.add_trace(
            go.Scatter(x=loss_output_df["error"], y=loss_output_df[loss_names[i-1]], mode='lines', name=loss_names[i-1], line=dict(color='black')),
            # px.line(loss_output_df, x=loss_output_df.columns[0], y=loss_names[i-1], title=loss_names[i-1]),
            row=row,
            col=col
        )

    # Update layout
    fig.update_layout(
        height=1000,  # Adjust height for better spacing
        width=1200,    # Adjust width for better spacing
        # title="7x3 Grid of Subplots",
        showlegend=False,  # Hide legend for simplicity
        margin=dict(l=5, r=5, t=30, b=5)
    )
    
    return fig

In [ ]:
losses_graph = draw_losses(loss_values, loss_names)
losses_graph.show()

In [ ]:
# losses_graph.write_image("plot.pdf", format="pdf")